Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [78]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [79]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [80]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [81]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [82]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.777323
Minibatch accuracy: 18.8%
Validation accuracy: 11.1%
Minibatch loss at step 50: 1.430789
Minibatch accuracy: 43.8%
Validation accuracy: 61.3%
Minibatch loss at step 100: 1.078670
Minibatch accuracy: 68.8%
Validation accuracy: 71.5%
Minibatch loss at step 150: 0.294682
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 200: 0.736642
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.293562
Minibatch accuracy: 62.5%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.342571
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.428158
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 400: 0.302040
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.725620
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.753031
Minibatch accuracy: 68.8%
Validation accuracy: 80.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 2 * image_size // 2 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer1_biases)
    hidden = conv
    conv = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer2_biases)
    hidden = conv
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [26]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.428610
Minibatch accuracy: 0.0%
Validation accuracy: 12.5%
Minibatch loss at step 50: 1.029826
Minibatch accuracy: 62.5%
Validation accuracy: 74.2%
Minibatch loss at step 100: 0.826169
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 150: 0.187352
Minibatch accuracy: 93.8%
Validation accuracy: 78.0%
Minibatch loss at step 200: 1.062095
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%
Minibatch loss at step 250: 1.208393
Minibatch accuracy: 62.5%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.367465
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 350: 0.523156
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.241400
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 450: 0.931319
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.639254
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [96]:
batch_size = 16
patch_size = 5
depth_l0 = num_channels
depth_l1 = 40
depth_l2 = 70
num_hidden = 128
num_hidden2 = 64

graph = tf.Graph()

with graph.as_default():
    
  # helper function for one layer
  def layer(tensor_in, weights, bias, conv_stride, pool_win,pool_stride):
    conv = tf.nn.conv2d(tensor_in, weights, [1, conv_stride, conv_stride, 1], padding='SAME')
    hidden = tf.nn.relu(conv + bias)
    return tf.nn.max_pool(hidden, ksize=[1, pool_win, pool_win, 1],
                        strides=[1, pool_stride, pool_stride, 1], padding='SAME')
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  keep_prob = tf.placeholder("float")
  layer0_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_l0], stddev=0.1))
  layer0_biases = tf.Variable(tf.zeros([depth_l0]))

  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_l0, depth_l1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_l1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_l1, depth_l2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_l2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 *depth_l2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  relu2_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden2], stddev=0.1))
  relu2_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden2]))  
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #conv = layer(data, layer0_weights, layer0_biases, 1, 1, 1)
    conv = layer(data, layer1_weights, layer1_biases, 1, 2, 2)
    conv = layer(conv, layer2_weights, layer2_biases, 1, 2, 2)
    shape = conv.get_shape().as_list()
    reshape = tf.reshape(conv, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.matmul(reshape, layer3_weights) + layer3_biases
    hidden = tf.nn.relu(hidden)
    hidden = tf.nn.dropout(hidden, keep_prob)
    hidden = tf.nn.relu(tf.matmul(hidden, relu2_weights) + relu2_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    0.1, global_step, 5000, 0.95, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 45001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval({keep_prob:1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 6.464273
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.286115
Minibatch accuracy: 6.2%
Validation accuracy: 12.4%
Minibatch loss at step 100: 2.290174
Minibatch accuracy: 18.8%
Validation accuracy: 16.4%
Minibatch loss at step 150: 1.436430
Minibatch accuracy: 37.5%
Validation accuracy: 44.6%
Minibatch loss at step 200: 1.976190
Minibatch accuracy: 37.5%
Validation accuracy: 55.2%
Minibatch loss at step 250: 1.284918
Minibatch accuracy: 56.2%
Validation accuracy: 71.5%
Minibatch loss at step 300: 1.074805
Minibatch accuracy: 56.2%
Validation accuracy: 74.0%
Minibatch loss at step 350: 0.719387
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 400: 0.435233
Minibatch accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss at step 450: 0.824557
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
Minibatch loss at step 500: 0.828591
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Min